In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

songs_data_path="F:\\new_downloads\\archive - 2025-06-04T000016.230\\Music Info.csv"
users_data_path="F:\\new_downloads\\archive - 2025-06-04T000016.230\\User Listening History.csv"


In [2]:
df_songs = pd.read_csv(songs_data_path)
print(df_songs.head())

print(df_songs.shape)
print(df_songs.info())

print(df_songs.isna().sum())

             track_id             name           artist  \
0  TRIOREW128F424EAF0   Mr. Brightside      The Killers   
1  TRRIVDJ128F429B0E8       Wonderwall            Oasis   
2  TROUVHL128F426C441  Come as You Are          Nirvana   
3  TRUEIND128F93038C4      Take Me Out  Franz Ferdinand   
4  TRLNZBD128F935E4D8            Creep        Radiohead   

                                 spotify_preview_url              spotify_id  \
0  https://p.scdn.co/mp3-preview/4d26180e6961fd46...  09ZQ5TmUG8TSL56n0knqrj   
1  https://p.scdn.co/mp3-preview/d012e536916c927b...  06UfBBDISthj1ZJAtX4xjj   
2  https://p.scdn.co/mp3-preview/a1c11bb1cb231031...  0keNu0t0tqsWtExGM3nT1D   
3  https://p.scdn.co/mp3-preview/399c401370438be4...  0ancVQ9wEcHVd0RrGICTE4   
4  https://p.scdn.co/mp3-preview/e7eb60e9466bc3a2...  01QoK9DA7VTeTSE3MNzp4I   

                                                tags genre  year  duration_ms  \
0  rock, alternative, indie, alternative_rock, in...   NaN  2004       222200   
1 

In [3]:
print((
    df_songs
    .isna()
    .mean()
    .sort_values(ascending=False)
    .head(2)
    .mul(100)
))

genre    55.906320
tags      2.223625
dtype: float64


In [4]:
print((
    df_songs
    .duplicated(subset="spotify_id")
    .sum()
))

9


In [5]:
df_songs.drop_duplicates(subset=["spotify_id","year","duration_ms"],inplace=True)

print((
    df_songs
    .duplicated(subset=["spotify_id","year","duration_ms"])
    .sum()
))

0


In [6]:
print(df_songs.head())

             track_id             name           artist  \
0  TRIOREW128F424EAF0   Mr. Brightside      The Killers   
1  TRRIVDJ128F429B0E8       Wonderwall            Oasis   
2  TROUVHL128F426C441  Come as You Are          Nirvana   
3  TRUEIND128F93038C4      Take Me Out  Franz Ferdinand   
4  TRLNZBD128F935E4D8            Creep        Radiohead   

                                 spotify_preview_url              spotify_id  \
0  https://p.scdn.co/mp3-preview/4d26180e6961fd46...  09ZQ5TmUG8TSL56n0knqrj   
1  https://p.scdn.co/mp3-preview/d012e536916c927b...  06UfBBDISthj1ZJAtX4xjj   
2  https://p.scdn.co/mp3-preview/a1c11bb1cb231031...  0keNu0t0tqsWtExGM3nT1D   
3  https://p.scdn.co/mp3-preview/399c401370438be4...  0ancVQ9wEcHVd0RrGICTE4   
4  https://p.scdn.co/mp3-preview/e7eb60e9466bc3a2...  01QoK9DA7VTeTSE3MNzp4I   

                                                tags genre  year  duration_ms  \
0  rock, alternative, indie, alternative_rock, in...   NaN  2004       222200   
1 

In [7]:
df_songs.reset_index(drop=True,inplace=True)

cols_to_remove = ["track_id","name","spotify_preview_url","spotify_id","genre"]


df_colab_filtering = df_songs.drop(columns=cols_to_remove)

print(df_colab_filtering)

                      artist  \
0                The Killers   
1                      Oasis   
2                    Nirvana   
3            Franz Ferdinand   
4                  Radiohead   
...                      ...   
50669            アンティック-珈琲店-   
50670                ACIDMAN   
50671               coldrain   
50672            アンティック-珈琲店-   
50673  9mm Parabellum Bullet   

                                                    tags  year  duration_ms  \
0      rock, alternative, indie, alternative_rock, in...  2004       222200   
1      rock, alternative, indie, pop, alternative_roc...  2006       258613   
2       rock, alternative, alternative_rock, 90s, grunge  1991       218920   
3      rock, alternative, indie, alternative_rock, in...  2004       237026   
4      rock, alternative, indie, alternative_rock, in...  2008       238640   
...                                                  ...   ...          ...   
50669                                                NaN  2008

In [8]:
print(df_colab_filtering.isna().sum())

artist                 0
tags                1126
year                   0
duration_ms            0
danceability           0
energy                 0
key                    0
loudness               0
mode                   0
speechiness            0
acousticness           0
instrumentalness       0
liveness               0
valence                0
tempo                  0
time_signature         0
dtype: int64


In [9]:
df_colab_filtering.fillna({"tags":"no_tags"},inplace=True)

print(df_colab_filtering.isna().sum().sum())

0


In [10]:

df_colab_filtering["artist"] = df_colab_filtering["artist"].str.lower()

In [11]:
print(
    df_songs
    .loc[:,'artist']
    .nunique()
)

print(df_songs.loc[:,'year'].nunique())

8317
75


In [12]:
print(
    df_songs
    .select_dtypes(include='number')
    .agg(['min','max'])
)

     year  duration_ms  danceability  energy  key  loudness  mode  \
min  1900         1439         0.000     0.0    0   -60.000     0   
max  2022      3816373         0.986     1.0   11     3.642     1   

     speechiness  acousticness  instrumentalness  liveness  valence    tempo  \
min        0.000         0.000             0.000     0.000    0.000    0.000   
max        0.954         0.996             0.999     0.999    0.993  238.895   

     time_signature  
min               0  
max               5  


In [13]:
print(
    df_songs
    .loc[:,'tags']
    .str.lower()
    .str.split(',')
    .explode()
    .str.strip()
    .value_counts()
    .loc[lambda ser: ser >= 1000]
)

tags
rock            10681
indie            7284
electronic       6592
alternative      6271
pop              4650
                ...  
ska              1088
gothic_metal     1072
grindcore        1040
french           1018
nu_metal         1006
Name: count, Length: 85, dtype: int64


In [14]:

from sklearn.preprocessing import MinMaxScaler,StandardScaler,OneHotEncoder
from category_encoders.count import CountEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer

In [15]:
print(df_colab_filtering.head(3))

print(df_colab_filtering.shape)

        artist                                               tags  year  \
0  the killers  rock, alternative, indie, alternative_rock, in...  2004   
1        oasis  rock, alternative, indie, pop, alternative_roc...  2006   
2      nirvana   rock, alternative, alternative_rock, 90s, grunge  1991   

   duration_ms  danceability  energy  key  loudness  mode  speechiness  \
0       222200         0.355   0.918    1    -4.360     1       0.0746   
1       258613         0.409   0.892    2    -4.373     1       0.0336   
2       218920         0.508   0.826    4    -5.783     0       0.0400   

   acousticness  instrumentalness  liveness  valence    tempo  time_signature  
0      0.001190          0.000000    0.0971    0.240  148.114               4  
1      0.000807          0.000000    0.2070    0.651  174.426               4  
2      0.000175          0.000459    0.0878    0.543  120.012               4  
(50674, 16)


In [16]:
frequency_encode_cols = ['year']
ohe_cols = ['artist',"time_signature","key"]
tfidf_col = 'tags'
standard_scale_cols = ["duration_ms","loudness","tempo"]
min_max_scale_cols = ["danceability","energy","speechiness","acousticness","instrumentalness","liveness","valence"]

In [17]:
len(frequency_encode_cols + ohe_cols + standard_scale_cols + min_max_scale_cols)

14

In [18]:
transformer = ColumnTransformer(transformers=[
    ("frequency_encode", CountEncoder(normalize=True,return_df=True), frequency_encode_cols),
    ("ohe", OneHotEncoder(handle_unknown="ignore"), ohe_cols),
    ("tfidf", TfidfVectorizer(max_features=85), tfidf_col),
    ("standard_scale", StandardScaler(), standard_scale_cols),
    ("min_max_scale", MinMaxScaler(), min_max_scale_cols)
],remainder='passthrough',n_jobs=-1,force_int_remainder_cols=False)

In [19]:
print(transformer)

transformer.fit(df_colab_filtering)

transformed_df=transformer.transform(df_colab_filtering)

print(transformed_df.shape)

print(transformed_df)

ColumnTransformer(force_int_remainder_cols=False, n_jobs=-1,
                  remainder='passthrough',
                  transformers=[('frequency_encode',
                                 CountEncoder(combine_min_nan_groups=True,
                                              normalize=True),
                                 ['year']),
                                ('ohe', OneHotEncoder(handle_unknown='ignore'),
                                 ['artist', 'time_signature', 'key']),
                                ('tfidf', TfidfVectorizer(max_features=85),
                                 'tags'),
                                ('standard_scale', StandardScaler(),
                                 ['duration_ms', 'loudness', 'tempo']),
                                ('min_max_scale', MinMaxScaler(),
                                 ['danceability', 'energy', 'speechiness',
                                  'acousticness', 'instrumentalness',
                                  'liven

In [20]:
from sklearn.metrics.pairwise import cosine_similarity
print(df_songs.loc[df_songs["artist"]=="Shakira"])

                 track_id                            name   artist  \
1025   TRLWDVU128F932B093              Whenever, Wherever  Shakira   
2068   TRILOWN128F426080F         Underneath Your Clothes  Shakira   
2205   TRXLMFJ12903CC06F7                        She Wolf  Shakira   
2469   TRSQAWU128F92EA20F                      La Tortura  Shakira   
3373   TRINUNP12903CD84D9                    Did It Again  Shakira   
4318   TROLUWR128F92E5858                  Te Dejo Madrid  Shakira   
4599   TRLNLES128F932DA8E                            Fool  Shakira   
4617   TRUIPZL12903CA0BFE                           Rules  Shakira   
6046   TRAAKDG128F42A0ECB                  Hips Don't Lie  Shakira   
6818   TRBAHID128F4278EAF               Objection (Tango)  Shakira   
7007   TRHEBDB128F92E9FC0                    Don't Bother  Shakira   
11904  TRADBSW128F933A7A8                            Loba  Shakira   
12305  TRYFVKK128F4240FE8                        Why Wait  Shakira   
12322  TROZGSI128F92

In [21]:
df_songs[df_songs["name"] == "Whenever, Wherever"]

,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
1025,TRLWDVU128F932B093,"Whenever, Wherever",Shakira,https://p.scdn.co/mp3-preview/09ddeb4ae33ee6e8...,07PHBDuUmOeZ7jeKSbAbKi,"rock, pop, female_vocalists, singer_songwriter...",NaN,2012,196826,0.787,...,1,-4.967,0,0.0474,0.298,0.000005,0.206,0.86,107.674,4


In [22]:
song_input = df_colab_filtering[df_songs["name"] == "Whenever, Wherever"]

print(song_input)

       artist                                               tags  year  \
1025  shakira  rock, pop, female_vocalists, singer_songwriter...  2012   

      duration_ms  danceability  energy  key  loudness  mode  speechiness  \
1025       196826         0.787   0.828    1    -4.967     0       0.0474   

      acousticness  instrumentalness  liveness  valence    tempo  \
1025         0.298          0.000005     0.206     0.86  107.674   

      time_signature  
1025               4  


In [23]:
input_vector = transformer.transform(song_input)

print(input_vector)


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 20 stored elements and shape (1, 8431)>
  Coords	Values
  (0, 0)	2012.0
  (0, 6253)	1.0
  (0, 8321)	1.0
  (0, 8324)	1.0
  (0, 8335)	0.48301799253775207
  (0, 8355)	0.45964030226253927
  (0, 8363)	0.37961149698832525
  (0, 8395)	0.37633989943093876
  (0, 8409)	0.2839886834338474
  (0, 8411)	0.4347852694755254
  (0, 8420)	-0.5049584637290775
  (0, 8421)	0.7308219064876488
  (0, 8422)	-0.5345609103611094
  (0, 8423)	0.7981744421906695
  (0, 8424)	0.828
  (0, 8425)	0.04968553459119497
  (0, 8426)	0.2991967871485944
  (0, 8427)	5.1551551551551555e-06
  (0, 8428)	0.20620620620620622
  (0, 8429)	0.8660624370594159


In [24]:
similarity_scores = cosine_similarity(transformed_df,input_vector)

print(similarity_scores.shape)

(50674, 1)


In [25]:
print(similarity_scores)

[[0.99999914]
 [0.99999847]
 [0.99999921]
 ...
 [0.99999877]
 [0.9999992 ]
 [0.99999891]]


In [26]:
top_10_songs_indexes = np.argsort(similarity_scores.ravel())[-11:][::-1]
print(top_10_songs_indexes)


[ 1025 12305  6046  6129 17241  6133  7172  6121  6526 38383  6287]


In [27]:
top_10_songs_names = df_songs.iloc[top_10_songs_indexes]
print(top_10_songs_names)

                 track_id                    name           artist  \
1025   TRLWDVU128F932B093      Whenever, Wherever          Shakira   
12305  TRYFVKK128F4240FE8                Why Wait          Shakira   
6046   TRAAKDG128F42A0ECB          Hips Don't Lie          Shakira   
6129   TRBAUVN128F932FEF8  Oops!...I Did It Again   Britney Spears   
17241  TRMBDIR128F4279C1F           Perfect Lover   Britney Spears   
6133   TRDXCSH128F92ED4A1            Bootylicious  Destiny's Child   
7172   TRLQBEN128F92E7415             Wild Things     Alessia Cara   
6121   TRGZIMZ128F930A016          La Isla Bonita          Madonna   
6526   TRXHWIA128E078A9BB            Cruel Summer       Bananarama   
38383  TRWUWRZ128F42ADA4A        Dreams for Plans          Shakira   
6287   TRZJJHG128EF34A3F9                Trick Me            Kelis   

                                     spotify_preview_url  \
1025   https://p.scdn.co/mp3-preview/09ddeb4ae33ee6e8...   
12305  https://p.scdn.co/mp3-preview/d7

In [28]:
def recommend(song_name, songs_data, transformed_data, k=10):
    """
    Recommends top k songs similar to the given song based on content-based filtering.

    Parameters:
    song_name (str): The name of the song to base the recommendations on.
    songs_data (DataFrame): The DataFrame containing song information.
    transformed_data (ndarray): The transformed data matrix for similarity calculations.
    k (int, optional): The number of similar songs to recommend. Default is 10.

    Returns:
    DataFrame: A DataFrame containing the top k recommended songs with their names, artists, and Spotify preview URLs.
    """
    # filter out the song from data
    song_row = songs_data.loc[songs_data["name"] == song_name,:]
    if song_row.empty:
        print("Song not found in the dataset.")
    else:
        # get the index of song
        song_index = song_row.index[0]
        print(song_index)
        # generate the input vector
        input_vector = transformed_data[song_index].reshape(1,-1)
        # calculate similarity scores
        similarity_scores = cosine_similarity(input_vector, transformed_data)
        print(similarity_scores.shape)
        # get the top k songs
        top_k_songs_indexes = np.argsort(similarity_scores.ravel())[-k-1:][::-1]
        print(top_k_songs_indexes)
        # get the top k songs names
        top_k_songs_names = songs_data.iloc[top_k_songs_indexes]
        # print the top k songs
        top_k_list = top_k_songs_names[['name','artist','spotify_preview_url']].reset_index(drop=True)
        return top_k_list

In [29]:
print(recommend("Whenever, Wherever",songs_data=df_songs,transformed_data=transformed_df,k=10))

1025
(1, 50674)
[ 1025 12305  6046  6129 17241  6133  7172  6121  6526 38383  6287]
                      name           artist  \
0       Whenever, Wherever          Shakira   
1                 Why Wait          Shakira   
2           Hips Don't Lie          Shakira   
3   Oops!...I Did It Again   Britney Spears   
4            Perfect Lover   Britney Spears   
5             Bootylicious  Destiny's Child   
6              Wild Things     Alessia Cara   
7           La Isla Bonita          Madonna   
8             Cruel Summer       Bananarama   
9         Dreams for Plans          Shakira   
10                Trick Me            Kelis   

                                  spotify_preview_url  
0   https://p.scdn.co/mp3-preview/09ddeb4ae33ee6e8...  
1   https://p.scdn.co/mp3-preview/d78c90c5cb5626be...  
2   https://p.scdn.co/mp3-preview/3859547944f57cfb...  
3   https://p.scdn.co/mp3-preview/7fb86827422540ad...  
4   https://p.scdn.co/mp3-preview/52671e54d36f077e...  
5   https://p.s